In [ ]:
# This R environment comes with all of CRAN preinstalled, as well as many other helpful packages
# The environment is defined by the kaggle/rstats docker image: https://github.com/kaggle/docker-rstats
# For example, here's several helpful packages to load in 

library(ggplot2) # Data visualization
library(readr) # CSV file I/O, e.g. the read_csv function

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

system("ls ../input")

# Any results you write to the current directory are saved as output.

In [ ]:
library(arules)
library(arulesViz)
library(data.table)
library(zoo)
library(forecast)
library(ggplot2)
test <- fread("../input/test.csv")
train <- fread("../input/train.csv")
store <- fread("../input/store.csv")

In [ ]:
str(train)


In [ ]:
str(store)

In [ ]:
summary(train)

In [ ]:
total <-merge(train, store, by="Store")
head(total)

In [ ]:
df <- subset(total, select = c(7, 9, 15, 4))

In [ ]:
str(df)

In [ ]:
head(df)

In [ ]:
# These Threshold i have according to mean value os sales
df$Sale[df$Sales < 2000] <- "low"
df$Sale[df$Sales >= 2000 & df$Sales < 7000] <- "Average"
df$Sale[df$Sales >= 7000] <- "High"

In [ ]:
head(df)

In [ ]:
df <- within(df, rm(Sales))
head(df)

In [ ]:
df <- as.data.frame(df)
df <- df[complete.cases(df),]
df$Promo <-as.factor(df$Promo)
df$SchoolHoliday <-as.factor(df$SchoolHoliday)
df$Promo2 <-as.factor(df$Promo2)
df$Sale <-as.factor(df$Sale)
str(df)
trans <- as(df, "transactions")
rules <- apriori(trans, parameter = list(supp = 0.090, conf = 0.15), appearance = list(rhs=c("Sale=Average", "Sale=High", "Sale=low"), 
                                     default="lhs"))

Total we have made 32 rules. These rules are dependent on support and confidence.These counting of these rules can be changed by changing the values of support and confidence by trial and error method.

**Support:** Support is the basic probability of an event to occur. If we have an event to buy product A, Support(A) is the number of transactions which includes A divided by total number of transactions.

**Confidence:** The confidence of an event is the conditional probability of the occurrence; the chances of A happening given B has already happened.

**Lift:** This is the ratio of confidence to expected confidence.The probability of all of the items in a rule occurring together (otherwise known as the support) divided by the product of the probabilities of the items on the left and right side occurring as if there was no association between them.

The lift value tells us how much better a rule is at predicting something than randomly guessing. The higher the lift, the stronger the association.

In [ ]:
rules <- sort(rules, by = "lift", decreasing='true')
head(inspect(rules))

In [ ]:
itemFrequencyPlot(trans, topN = 9)

The above visulization shows the frequency of item set like frequency of Promo2 is more than Promo etc.

In [ ]:
plot(rules[1:20],
method = "graph",
control = list(type = "rules"))

This representation gives us a graph model of items in our dataset.

The size of graph nodes is based on support levels and the colour on lift ratios. The incoming lines show the Antecedants or the LHS and the RHS is represented by names of items

The above graph shows us that most of our transactions were consolidated around **Promo** and .**Sale = high** . This means that when we doing **Promo*** our sale will go high. But in previous visulazation we have done less **Promo** than **Promo2**. So, we can increase **Promo** ,sale will go high.